In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Duchenne_Muscular_Dystrophy"
cohort = "GSE48828"

# Input paths
in_trait_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy"
in_cohort_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy/GSE48828"

# Output paths
out_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/GSE48828.csv"
out_gene_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE48828.csv"
out_clinical_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/clinical_data/GSE48828.csv"
json_path = "../../output/preprocess/Duchenne_Muscular_Dystrophy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression and Splicing Analysis of Myotonic Dystrophy and Other Dystrophic Muscle"
!Series_summary	"The prevailing patho-mechanistic paradigm for myotonic dystrophy (DM) is that the aberrant presence of embryonic isoforms is responsible for many, if not most, aspects of the pleiotropic disease phenotype.  In order to identify such aberrantly expressed isoforms in skeletal muscle of DM type 1 (DM1) and type 2 (DM2) patients, we utilized the Affymetrix exon array to characterize the largest collection of DM samples analyzed to date, and included non-DM dystrophic muscle samples (NMD) as disease controls."
!Series_overall_design	"For the exon array profiling on the Human Exon 1.0 ST array (Affymetrix Santa Clara, CA) we used a panel of 28 skeletal muscle biopsies from DM1 (n=8), DM2 (n=10), Becker muscular dystrophy, BMD, (n=3), Duchenne muscular dystrophy, DMD (n=1), Tibial muscular dystrophy, TMD, (n=2) and normal skeletal muscle (n=4).  Normal co

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Analyzing gene expression data availability
is_gene_available = True  # The dataset contains gene expression data on the Affymetrix Human Exon 1.0 ST array

# Analyzing trait data availability
# From the sample characteristics dictionary, key 0 contains diagnosis information including DMD
trait_row = 0  # The key for trait data in sample characteristics dictionary

# Age data availability
age_row = 2  # The key for age data in sample characteristics dictionary

# Gender data availability
gender_row = 1  # The key for gender data in sample characteristics dictionary

# Define conversion functions for each variable
def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 for non-DMD, 1 for DMD)."""
    if value is None or pd.isna(value):
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates Duchenne Muscular Dystrophy
    if 'Duchenne Muscular Dystrophy' in value:
        return 1
    else:
        return 0

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous numeric value."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Handle various age formats and missing values
    if value in ['Not available', 'na', '']:
        return None
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender values
    if value.upper() in ['F', 'FEMALE']:
        return 0
    elif value.upper() in ['M', 'MALE']:
        return 1
    else:  # 'Not available' or other values
        return None

# Validate and save cohort info (initial filtering)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Print a message about the results of the initial validation
print(f"Initial validation - Gene data available: {is_gene_available}, Trait data available: {is_trait_available}")


Initial validation - Gene data available: True, Trait data available: True


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Matrix table marker not found in first 100 lines
Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers look like probe IDs from a microarray platform rather than human gene symbols
# They are numeric IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for mapping
# The 'ID' column in gene_annotation contains the same identifiers as in gene_data (probe IDs)
# The 'gene_assignment' column contains gene symbol information, but in a complex format

# Let's examine the structure of the gene_assignment field more closely
print("Sample gene_assignment values:")
non_empty_assignments = gene_annotation['gene_assignment'].dropna().replace('---', None).dropna().head(3)
for idx, assignment in enumerate(non_empty_assignments):
    print(f"Example {idx+1}: {assignment}")

# 2. Define a more specific extraction function for this dataset
def extract_gene_symbols_from_assignment(assignment_text):
    """Extract gene symbols from complex gene_assignment text format."""
    if assignment_text is None or pd.isna(assignment_text) or assignment_text == '---':
        return []
    
    # Split by /// to get separate gene entries
    gene_entries = assignment_text.split('///')
    symbols = []
    
    for entry in gene_entries:
        # Split each entry by // and extract the second element (gene symbol)
        parts = entry.strip().split('//')
        if len(parts) >= 2:
            symbol = parts[1].strip()
            if symbol and symbol != '---':
                symbols.append(symbol)
    
    return symbols

# 3. Create a modified version of the mapping dataframe with extracted gene symbols
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
mapping_df['Gene'] = mapping_df['gene_assignment'].apply(extract_gene_symbols_from_assignment)
mapping_df = mapping_df.explode('Gene').dropna(subset=['Gene'])
mapping_df = mapping_df[['ID', 'Gene']]

# Check the resulting mapping dataframe
print("\nImproved gene mapping preview (first 10 rows):")
print(preview_df(mapping_df.head(10)))

# 4. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Check the results - print the first few gene symbols and their expression values
print("\nGene expression data after mapping (first 5 genes):")
print(preview_df(gene_data.head(5)))

# Check dimensions of the resulting dataframe
print(f"\nDimensions of gene expression data: {gene_data.shape}")

# Save the gene expression data for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Sample gene_assignment values:
Example 1: NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771
Example 2: NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 // olfactory receptor, family 4, subfamily F, member 5 // 1p36.33 // 79501 /// ENST00000318050 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// ENST00000335137 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// ENST00000326183 // OR4F5 // ol


Improved gene mapping preview (first 10 rows):
{'ID': ['2315100', '2315100', '2315100', '2315100', '2315125'], 'Gene': ['DDX11L2', 'DDX11L9', 'DDX11L2', 'DDX11L2', 'OR4F17']}



Gene expression data after mapping (first 5 genes):
{'GSM1185313': [26.80524, 6.70131, 52.586949999999995, 32.1961, 36.43848], 'GSM1185314': [25.04744, 6.26186, 48.33504, 30.528550000000003, 39.52976], 'GSM1185315': [26.10308, 6.52577, 50.88642, 30.96505, 38.271], 'GSM1185316': [23.97268, 5.99317, 50.92048, 30.78035, 36.27584], 'GSM1185317': [24.65856, 6.16464, 48.13991, 30.1939, 39.51208], 'GSM1185318': [28.12448, 7.03112, 59.186659999999996, 34.2899, 35.71308], 'GSM1185319': [25.6806, 6.42015, 47.675290000000004, 31.274, 37.94328], 'GSM1185320': [27.04864, 6.76216, 55.54588, 32.52645, 37.22852], 'GSM1185321': [25.77264, 6.44316, 52.70629, 31.888299999999997, 37.9546], 'GSM1185322': [24.53292, 6.13323, 53.968070000000004, 30.308500000000002, 37.40376], 'GSM1185323': [27.44768, 6.86192, 59.14688, 32.926950000000005, 36.83836], 'GSM1185324': [26.79176, 6.69794, 51.47896, 31.3275, 36.5818], 'GSM1185325': [24.84228, 6.21057, 49.93586, 30.87975, 38.5736], 'GSM1185326': [24.4734, 6.11835, 

Gene expression data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE48828.csv
